# 第9章:  RNN, CNN
https://nlp100.github.io/ja/ch09.html

## 80. ID番号への変換

問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に`1`，2番目に頻出する単語に`2`，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて`0`とせよ．

In [76]:
import pandas as pd
import texthero as hero
from sklearn.feature_extraction.text import CountVectorizer

df_train = pd.read_table('train.txt', header=None)
title = hero.clean(df_train[1])
words = sum(title.str.split().values, [])

# ユニークな単語の個数を数えて，降順に並べてidxをidにしている
count = pd.Series(words).value_counts()
count = count[count>1] 
ids_dict = count.map({c:i+1 for i, c in enumerate(count.unique())})


In [96]:
# wordsは配列かつlower適用済みを仮定
def words2ids(words, ids_dict):
    return [ids_dict.get(word, 0) for word in words]

# テスト
ids = words2ids(title[0].lower().split(), ids_dict)
ids

[131, 129, 141, 124, 131, 139, 0, 135, 3, 158]

## 81. RNNによる予測
ID番号で表現された単語列$\boldsymbol{x} = (x_1, x_2, \dots, x_T)$がある．ただし，$T$は単語列の長さ，$x_t \in \mathbb{R}^{V}$は単語のID番号のone-hot表記である（$V$は単語の総数である）．再帰型ニューラルネットワーク（RNN: Recurrent Neural Network）を用い，単語列xからカテゴリ$y$を予測するモデルとして，次式を実装せよ．

$$
\overrightarrow{h}_0 = 0, \\
\overrightarrow{h}_t = {\rm \overrightarrow{RNN}}(\mathrm{emb}(x_t), \overrightarrow{h}_{t-1}), \\
y = {\rm softmax}(W^{(yh)} \overrightarrow{h}_T + b^{(y)})
$$

ただし，$\mathrm{emb}(x) \in \mathbb{R}^{d_w}$は単語埋め込み（単語のone-hot表記から単語ベクトルに変換する関数），$\overrightarrow{h}_t \in \mathbb{R}^{d_h}$は時刻$t$の隠れ状態ベクトル，${\rm \overrightarrow{RNN}}(x,h)$は入力$x$と前時刻の隠れ状態$h$から次状態を計算するRNNユニット，$W^{(yh)} \in \mathbb{R}^{L \times d_h}$は隠れ状態ベクトルからカテゴリを予測するための行列，$b^{(y)} \in \mathbb{R}^{L}$はバイアス項である（$d_w, d_h, L$)はそれぞれ，単語埋め込みの次元数，隠れ状態ベクトルの次元数，ラベル数である）．RNNユニット${\rm \overrightarrow{RNN}}(x,h)$には様々な構成が考えられるが，典型例として次式が挙げられる．

$$
{\rm \overrightarrow{RNN}}(x,h) = g(W^{(hx)} x + W^{(hh)}h + b^{(h)})
$$

ただし，$W^{(hx)} \in \mathbb{R}^{d_h \times d_w}，W^{(hh)} \in \mathbb{R}^{d_h \times d_h}, b^{(h)} \in \mathbb{R}^{d_h}$はRNNユニットのパラメータ，$g$は活性化関数（例えばtanhやReLUなど）である．

なお，この問題ではパラメータの学習を行わず，ランダムに初期化されたパラメータでyを計算するだけでよい．次元数などのハイパーパラメータは，$d_w=300,d_h=50$など，適当な値に設定せよ（以降の問題でも同様である）．

In [177]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, dw, dh, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size+1, dw)
        self.rnn = nn.RNN(dw, dh, batch_first=True)
        self.linear = nn.Linear(dh, 4)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x, hidden=None):
        x = self.emb(x)
        y, _ = self.rnn(x, hidden)
        y = self.linear(y[:, -1, :])
        # y = self.softmax(y)
        return y

model = RNN(dw=300, dh=50, vocab_size=len(ids_dict))
model(torch.tensor(ids)[None]) # テスト

tensor([[ 0.7829,  0.9514,  0.2024, -0.7470]], grad_fn=<AddmmBackward0>)

## 82. 確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．


In [178]:
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Xはpd.Seriesで文章を想定
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None, max_len=30):
        self.X = hero.clean(X).str.split()
        self.y = y
        self.max_len = max_len

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        x, y = words2ids(self.X[index], ids_dict), self.y[index]
        x = self.padding(ids)
        return torch.tensor(x), torch.tensor(y).long()
    
    def padding(self , ids):
        if len(ids) < self.max_len:
            ids += [0]*(self.max_len - len(ids))
        else:
            ids = ids[:self.max_len]
        return ids

def train_epoch(model, train_dataloader, criterion, optimizer, device):
    model.to(device)
    model.train()
    train_loss = 0
    train_accuracy = 0
    for X, T in train_dataloader:
        X, T = X.to(device), T.to(device)
        Y = model(X)
        loss = criterion(Y, T)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = Y.argmax(1)
        train_loss += loss.item()*len(X)/len(train_dataloader.dataset)
        train_accuracy += torch.sum(pred == T)/len(train_dataloader.dataset)
    return train_loss, train_accuracy

def valid_epoch(model, valid_dataloader, criterion, device):
    model.to(device)
    model.eval()
    valid_loss = 0
    valid_accuracy = 0
    for X, T in valid_dataloader:
        X, T = X.to(device), T.to(device)
        with torch.no_grad():
            Y = model(X)
            loss = criterion(Y, T)
        
        pred = Y.argmax(1)
        valid_loss += loss.item()*len(X)/len(valid_dataloader.dataset)
        valid_accuracy += torch.sum(pred == T)/len(valid_dataloader.dataset)
    return valid_loss, valid_accuracy

In [179]:
category_dict = {
    'b': 0,
    't': 1,
    'e': 2,
    'm': 3,
}

def get_feature_and_label(txt_path):
    df = pd.read_table(txt_path, header=None, names=['category', 'title'])
    df['label'] = df['category'].map(category_dict)
    return df['title'], df['label'].values

X_train, y_train = get_feature_and_label('train.txt')
X_valid, y_valid = get_feature_and_label('valid.txt')
X_test, y_test = get_feature_and_label('test.txt')

In [190]:
device = torch.device('cpu')

dw, dh = 300, 50
max_len = 100
num_epochs = 5
batch_size = 64
lr = 1e-1

model = RNN(dw=dw, dh=dh, vocab_size=len(ids_dict))
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train_dataloader = DataLoader(Dataset(X_train, y_train, max_len=max_len), batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(Dataset(X_valid, y_valid, max_len=max_len), batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(Dataset(X_test, y_test, max_len=max_len), batch_size=batch_size, shuffle=False)

In [191]:
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')

epoch: 1, train_loss: 1.183, train_accuracy:  0.409. valid_loss:  1.205, valid_accuracy:  0.422
epoch: 2, train_loss: 1.175, train_accuracy:  0.410. valid_loss:  1.163, valid_accuracy:  0.422
epoch: 3, train_loss: 1.172, train_accuracy:  0.408. valid_loss:  1.175, valid_accuracy:  0.396
epoch: 4, train_loss: 1.171, train_accuracy:  0.403. valid_loss:  1.167, valid_accuracy:  0.396
epoch: 5, train_loss: 1.171, train_accuracy:  0.412. valid_loss:  1.170, valid_accuracy:  0.422


## 83. ミニバッチ化・GPU上での学習
問題82のコードを改変し，$B$事例ごとに損失・勾配を計算して学習を行えるようにせよ（$B$の値は適当に選べ）．また，GPU上で学習を実行せよ．


In [221]:
batch_size = 64

train_dataloader = DataLoader(Dataset(X_train, y_train, max_len=max_len), batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(Dataset(X_valid, y_valid, max_len=max_len), batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(Dataset(X_test, y_test, max_len=max_len), batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')

epoch: 1, train_loss: 1.169, train_accuracy:  0.402. valid_loss:  1.175, valid_accuracy:  0.396
epoch: 2, train_loss: 1.167, train_accuracy:  0.412. valid_loss:  1.178, valid_accuracy:  0.396
epoch: 3, train_loss: 1.167, train_accuracy:  0.407. valid_loss:  1.163, valid_accuracy:  0.422
epoch: 4, train_loss: 1.166, train_accuracy:  0.414. valid_loss:  1.172, valid_accuracy:  0.396
epoch: 5, train_loss: 1.166, train_accuracy:  0.416. valid_loss:  1.165, valid_accuracy:  0.396


## 84. 単語ベクトルの導入
事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での[学習済み単語ベクトル](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing)）で単語埋め込みemb(x)を初期化し，学習せよ．

In [225]:
class RNN(nn.Module):
    def __init__(self, dw, dh, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size+1, dw)
        self.rnn = nn.RNN(dw, dh, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(dh*2, 4)
        self.softmax = nn.Softmax(dim=-1)

    def init_emb(self, vectors, vocab_list):
        for i, token in enumerate(vocab_list):
            if token in vectors:
                self.emb.weight.data[i] = torch.from_numpy(vectors[token])

    def forward(self, x, hidden=None):
        x = self.emb(x)
        y, hidden = self.rnn(x, hidden)
        y = torch.cat([hidden[0], hidden[1]], dim=1)
        y = self.linear(y[:, -1, :])
        # y = self.softmax(y)
        return y


In [ ]:
from gensim.models import KeyedVectors
vectors = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)
model = RNN(dw=dw, dh=dh, vocab_size=len(ids_dict))
model.init_emb(vectors, words)
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')

## 85. 双方向RNN・多層化
順方向と逆方向のRNNの両方を用いて入力テキストをエンコードし，モデルを学習せよ．

$$
\overleftarrow{h}_{T+1} = 0, \\
\overleftarrow{h}_t = {\rm \overleftarrow{RNN}}(\mathrm{emb}(x_t), \overleftarrow{h}_{t+1}), \\
y = {\rm softmax}(W^{(yh)} [\overrightarrow{h}_T; \overleftarrow{h}_1] + b^{(y)})
$$

ただし，$\overrightarrow{h}_t \in \mathbb{R}^{d_h}, \overleftarrow{h}_t \in \mathbb{R}^{d_h}$はそれぞれ，順方向および逆方向のRNNで求めた時刻$t$の隠れ状態ベクトル，${\rm \overleftarrow{RNN}}(x,h)$は入力$x$と次時刻の隠れ状態$h$から前状態を計算するRNNユニット，$W^{(yh)} \in \mathbb{R}^{L \times 2d_h}$は隠れ状態ベクトルからカテゴリを予測するための行列，$b^{(y)} \in \mathbb{R}^{L}$はバイアス項である．また，$[a;b]$はベクトル$a$と$b$の連結を表す。

さらに，双方向RNNを多層化して実験せよ．

In [220]:
class RNN(nn.Module):
    def __init__(self, dw, dh, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size+1, dw)
        self.rnn = nn.RNN(dw, dh, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(dh*2, 4)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x, hidden=None):
        x = self.emb(x)
        y, hidden = self.rnn(x, hidden)
        y = torch.cat([hidden[0], hidden[1]], dim=1)
        y = self.linear(y[:, -1, :])
        # y = self.softmax(y)
        return y

model = RNN(dw=dw, dh=dh, vocab_size=len(ids_dict))
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')

## 86. 畳み込みニューラルネットワーク (CNN)
ID番号で表現された単語列$\boldsymbol{x} = (x_1, x_2, \dots, x_T)$がある．ただし，$T$は単語列の長さ，$x_t \in \mathbb{R}^{V}$は単語のID番号のone-hot表記である（$V$は単語の総数である）．畳み込みニューラルネットワーク（CNN: Convolutional Neural Network）を用い，単語列$\boldsymbol{x}$からカテゴリ$y$を予測するモデルを実装せよ．

ただし，畳み込みニューラルネットワークの構成は以下の通りとする．

- 単語埋め込みの次元数: $d_w$
- 畳み込みのフィルターのサイズ: 3 トークン
- 畳み込みのストライド: 1 トークン
- 畳み込みのパディング: あり
- 畳み込み演算後の各時刻のベクトルの次元数: $d_h$
- 畳み込み演算後に最大値プーリング（max pooling）を適用し，入力文を$d_h$次元の隠れベクトルで表現

すなわち，時刻$t$の特徴ベクトル$pt∈ℝdh$は次式で表される．

$$
p_t = g(W^{(px)} [\mathrm{emb}(x_{t-1}); \mathrm{emb}(x_t); \mathrm{emb}(x_{t+1})] + b^{(p)})
$$

ただし，$W^{(px)} \in \mathbb{R}^{d_h \times 3d_w}, b^{(p)} \in \mathbb{R}^{d_h}$はCNNのパラメータ，$g$は活性化関数（例えばtanhやReLUなど），$[a;b;c]$はベクトル$a,b,c$の連結である．なお，行列$W^{(px)}$の列数が3$d_w$になるのは，3個のトークンの単語埋め込みを連結したものに対して，線形変換を行うためである．

最大値プーリングでは，特徴ベクトルの次元毎に全時刻における最大値を取り，入力文書の特徴ベクトル$c \in \mathbb{R}^{d_h}$を求める．$c[i]$でベクトル$c$の$i$番目の次元の値を表すことにすると，最大値プーリングは次式で表される．

$$
c[i] = \max_{1 \leq t \leq T} p_t[i]
$$

最後に，入力文書の特徴ベクトル$c$に行列$W^{(yc)} \in \mathbb{R}^{L \times d_h}$とバイアス項$b^{(y)}\in \mathbb{R}^L$による線形変換とソフトマックス関数を適用し，カテゴリ$y$を予測する．
なお，この問題ではモデルの学習を行わず，ランダムに初期化された重み行列で$y$を計算するだけでよい．

In [227]:
class CNN(nn.Module):
    def __init__(self, dw, dh, vocab_size):
        super().__init__()
        self.emb = nn.Embedding(vocab_size+1, dw)
        self.conv = nn.Conv1d(dw, dh, 3, padding=1) 
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(max_len)
        self.linear = nn.Linear(dh, 4)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], x.shape[2], x.shape[1])
        x = self.conv(x)
        x = self.relu(x)
        x = x.view(x.shape[0], x.shape[1], x.shape[2])
        x = self.pool(x)
        x = x.view(x.shape[0], x.shape[1])
        y = self.linear(x)
        # y = self.softmax(y)
        return y

## 87. 確率的勾配降下法によるCNNの学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題86で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr)
model = CNN(dw=dw, dh=dh, vocab_size=len(ids_dict))

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')

## 88. パラメータチューニング
問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．


In [324]:
# Adamに変更
optimizer = optim.Adam(model.parameters(), lr=lr)
model = CNN(dw=dw, dh=dh, vocab_size=len(ids_dict))

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')

## 89. 事前学習済み言語モデルからの転移学習
事前学習済み言語モデル（例えばBERTなど）を出発点として，ニュース記事見出しをカテゴリに分類するモデルを構築せよ．

In [306]:
from transformers import BertModel, BertTokenizer

class BERT(nn.Module):
    def __init__(self, n_unit=768, n_classes=4):
        super().__init__()
        self.n_unit = n_unit
        self.bert_model = BertModel.from_pretrained('bert-base-uncased') 
        self.fc = nn.Linear(n_unit, n_classes)

    def forward(self, ids):
        seg_ids = torch.zeros_like(ids)
        attention_mask = (ids > 0)
        last_hidden_state = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask).last_hidden_state
        x = last_hidden_state[:, 0, :]
        logit = self.fc(x.view(-1, self.n_unit))
        return logit


# Xはpd.Seriesで文章を想定
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y=None, max_len=30, tokenizer=None):
        self.X = hero.clean(X)
        self.y = y
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        x, y = tokenizer.encode(self.X[index], max_length=512), self.y[index]
        x = self.padding(x)
        return torch.tensor(x), torch.tensor(y).long()

    def padding(self , ids):
        if len(ids) < self.max_len:
            ids += [0]*(self.max_len - len(ids))
        else:
            ids = ids[:self.max_len]
        return ids

In [307]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataloader = DataLoader(Dataset(X_train, y_train, max_len=max_len, tokenizer=tokenizer), batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(Dataset(X_valid, y_valid, max_len=max_len, tokenizer=tokenizer), batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(Dataset(X_test, y_test, max_len=max_len, tokenizer=tokenizer), batch_size=batch_size, shuffle=False)

In [323]:
optimizer = optim.SGD(model.parameters(), lr=lr)
model = BERT()

for name, param in model.named_parameters():
    if 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [322]:
for epoch in range(num_epochs):
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    valid_loss, valid_accuracy = valid_epoch(model, valid_dataloader, criterion, device)
    print(f'epoch: {epoch+1}, train_loss: {train_loss:.3f}, train_accuracy: {train_accuracy: .3f}. valid_loss: {valid_loss: .3f}, valid_accuracy: {valid_accuracy: .3f}')